In [1]:
from dataclasses import replace
from pydoc import doc
import pandas as pd
import os

# Read data into papers
papers = pd.read_csv('./digestItems.csv')

papers = papers.drop(columns=['category', 'date', 'publication', 
                  'links-within-blurb', 'states'], axis=1)

# Load the regular expression library
import re
# Remove punctuation
papers['blurb_text_processed'] = \
papers['blurb'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
papers['blurb_text_processed'] = \
papers['blurb_text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
papers['blurb_text_processed'].head()

import gensim
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

data = papers.blurb_text_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
# print(data_words[:1][0][:30])


import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
# print(corpus[:1][0][:30])

from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaModel(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
# pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print(doc_lda)

In [5]:
import pickle 
import pyLDAvis.gensim_models
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./docs/ldavis_prepared_'+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself

LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
with open(LDAvis_data_filepath, 'wb') as f:
    pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './docs/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

/usr/local/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.106707 -0.015684       1        1  14.261941
3     -0.099126 -0.177770       2        1  13.181993
4     -0.117030  0.021090       3        1  10.542462
8      0.069199 -0.026162       4        1   9.910699
1     -0.087121  0.167924       5        1   9.808425
5     -0.104537  0.025487       6        1   9.491338
0      0.028515 -0.025687       7        1   9.449424
7      0.138220 -0.004918       8        1   9.199755
6      0.094147  0.065559       9        1   9.141874
2     -0.028973 -0.029838      10        1   5.012089, topic_info=          Term       Freq      Total Category  logprob  loglift
359   campaign  22.000000  22.000000  Default  30.0000  30.0000
217    climate  28.000000  28.000000  Default  29.0000  29.0000
6      federal  40.000000  40.000000  Default  28.0000  28.0000
561  tennessee  12.000000  12.000000  Default  27.0000  27.0000
62      energy  98.000000  98.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
455  advocates   1.708001  16.205163  Topic10  -5.1769   0.7433
164    leaders   1.671571  12.215094  Topic10  -5.1984   1.0044
118      solar   1.729944  29.763318  Topic10  -5.1641   0.1481
763      homes   1.599242   7.064671  Topic10  -5.2426   1.5077
14       plant   1.604548  13.948554  Topic10  -5.2393   0.8308

[628 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
201       1  0.734519   acre
201       9  0.122420   acre
201      10  0.122420   acre
714       1  0.420389  acres
714       3  0.105097  acres
...     ...       ...    ...
398       5  0.064720   york
398       6  0.194161   york
398       8  0.064720   york
398      10  0.064720   york
940       7  0.706114   yost

[1162 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 4, 5, 9, 2, 6, 1, 8, 7, 3])